In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dikshaadke/motionblurdataset")

print("Path to dataset files:", path)

100%|██████████| 3.87G/3.87G [00:42<00:00, 98.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/dikshaadke/motionblurdataset/versions/1


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import os
import numpy as np
from PIL import Image
import kagglehub

# Download Kaggle dataset
path = kagglehub.dataset_download("dikshaadke/motionblurdataset")
print("Path to dataset files:", path)

# Set paths
base_dir = "/root/.cache/kagglehub/datasets/dikshaadke/motionblurdataset/versions/1/content/clean_dataset"
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# Debug: Print directory structure
print("\nDataset structure:")
print("Base dir contents:", os.listdir(base_dir))
if os.path.exists(train_dir):
    print("Train dir contents:", os.listdir(train_dir))
else:
    print("Train dir does not exist:", train_dir)
if os.path.exists(test_dir):
    print("Test dir contents:", os.listdir(test_dir))
else:
    print("Test dir does not exist:", test_dir)

# Image parameters
IMG_HEIGHT = 128
IMG_WIDTH = 128
BATCH_SIZE = 32
NUM_CLASSES = 2

# Custom preprocessing function to handle varying image sizes
def preprocess_image(image):
    # Convert tensor to PIL Image
    image = tf.keras.preprocessing.image.array_to_img(image)

    # Resize while preserving aspect ratio
    image = image.resize((IMG_WIDTH, IMG_HEIGHT), Image.LANCZOS)

    # Convert back to array and normalize
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = image / 255.0  # Normalize to [0, 1]

    return image

# Load and preprocess dataset
def load_dataset():
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=preprocess_image,
        validation_split=0.0  # No validation split since test folder is separate
    )

    train_generator = datagen.flow_from_directory(
        train_dir,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='binary',
        classes=['blur', 'non-blur']  # Updated class names
    )

    test_generator = datagen.flow_from_directory(
        test_dir,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='binary',
        classes=['blur', 'non-blur']  # Updated class names
    )

    return train_generator, test_generator


Path to dataset files: /root/.cache/kagglehub/datasets/dikshaadke/motionblurdataset/versions/1

Dataset structure:
Base dir contents: ['train', 'test']
Train dir contents: ['non-blur', 'blur']
Test dir contents: ['non-blur', 'blur']


In [ ]:

# Build deeper CNN model
def build_model():
    model = models.Sequential([
        layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        layers.Conv2D(16, 3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(16, 3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2),
        layers.Conv2D(32, 3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(32, 3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2),
        layers.Conv2D(64, 3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, 3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2),
        layers.Conv2D(128, 3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')  # Binary classification
    ])
    return model

# Train the model
def train_model():
    train_generator, test_generator = load_dataset()

    model = build_model()
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    model.summary()

    history = model.fit(
        train_generator,
        validation_data=test_generator,
        epochs=50,
        steps_per_epoch=train_generator.samples // BATCH_SIZE,
        validation_steps=test_generator.samples // BATCH_SIZE
    )

    return model

# Convert to TFLite
def convert_to_tflite(model):
    # Save the model
    model.save('blur_classifier.h5')

    # Convert to TFLite with quantization
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_types = [tf.float16]  # Reduce model size
    tflite_model = converter.convert()

    # Save TFLite model
    with open('blur_classifier.tflite', 'wb') as f:
        f.write(tflite_model)

    print("TFLite model saved as 'blur_classifier.tflite'")

# Main execution
if __name__ == "__main__":
    model = train_model()
    convert_to_tflite(model)

Found 31649 images belonging to 2 classes.
Found 1667 images belonging to 2 classes.


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_35 (Conv2D)              │ (None, 128, 128, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_40          │ (None, 128, 128, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_36 (Conv2D)              │ (None, 128, 128, 16)   │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_41          │ (None, 128, 128, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 64, 64, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_37 (Conv2D)              │ (None, 64, 64, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_42          │ (None, 64, 64, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_38 (Conv2D)              │ (None, 64, 64, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_43          │ (None, 64, 64, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_21 (MaxPooling2D) │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_39 (Conv2D)              │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_44          │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_40 (Conv2D)              │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_45          │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_22 (MaxPooling2D) │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_41 (Conv2D)              │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_46          │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_23 (MaxPooling2D) │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 256)            │     2,097,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_47          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │             

 Total params: 2,278,801 (8.69 MB)

 Trainable params: 2,277,585 (8.69 MB)

 Non-trainable params: 1,216 (4.75 KB)

Epoch 1/50
989/989 ━━━━━━━━━━━━━━━━━━━━ 194s 185ms/step - accuracy: 0.8777 - loss: 0.3313 - val_accuracy: 0.8876 - val_loss: 0.2691
Epoch 2/50
  1/989 ━━━━━━━━━━━━━━━━━━━━ 39s 40ms/step - accuracy: 0.9062 - loss: 0.1740

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


989/989 ━━━━━━━━━━━━━━━━━━━━ 17s 17ms/step - accuracy: 0.9062 - loss: 0.1740 - val_accuracy: 0.8876 - val_loss: 0.2722
Epoch 3/50
989/989 ━━━━━━━━━━━━━━━━━━━━ 215s 160ms/step - accuracy: 0.9398 - loss: 0.1827 - val_accuracy: 0.9405 - val_loss: 0.1680
Epoch 4/50
989/989 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.9375 - loss: 0.0999 - val_accuracy: 0.9411 - val_loss: 0.1662
Epoch 5/50
989/989 ━━━━━━━━━━━━━━━━━━━━ 202s 167ms/step - accuracy: 0.9440 - loss: 0.1604 - val_accuracy: 0.9177 - val_loss: 0.2166
Epoch 6/50
989/989 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 1.0000 - loss: 0.0442 - val_accuracy: 0.9207 - val_loss: 0.2092
Epoch 7/50
989/989 ━━━━━━━━━━━━━━━━━━━━ 182s 154ms/step - accuracy: 0.9518 - loss: 0.1422 - val_accuracy: 0.9303 - val_loss: 0.2235
Epoch 8/50
989/989 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.9375 - loss: 0.1502 - val_accuracy: 0.9207 - val_loss: 0.2369
Epoch 9/50
989/989 ━━━━━━━━━━━━━━━━━━━━ 202s 161ms/step - accuracy: 0.9531 - loss: 0.1365 - val_accur

Saved artifact at '/tmp/tmpbsgety22'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor_130')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  138640418715216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138640418716752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138640418717328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138640418715792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138640418714448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138640418716560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138640418715024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138640419128720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138640419127952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138640419127568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1386404191

In [ ]:
def load_test_dataset():
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=preprocess_image
    )
    test_generator = datagen.flow_from_directory(
        test_dir,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='binary',
        classes=['blur', 'non-blur'],
        shuffle=False
    )
    return test_generator

# Evaluate Keras model
def evaluate_keras_model():
    model = tf.keras.models.load_model('blur_classifier.h5')
    test_generator = load_test_dataset()
    loss, accuracy = model.evaluate(test_generator)
    print(f"Keras Model - Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Evaluate TFLite model
def evaluate_tflite_model():
    test_generator = load_test_dataset()
    interpreter = tf.lite.Interpreter(model_path='blur_classifier.tflite')
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    correct = 0
    total = 0
    for images, labels in test_generator:
        for i in range(len(images)):
            interpreter.set_tensor(input_details[0]['index'], images[i:i+1])
            interpreter.invoke()
            prediction = interpreter.get_tensor(output_details[0]['index'])
            predicted_label = 1 if prediction[0] > 0.5 else 0
            if predicted_label == int(labels[i]):
                correct += 1
            total += 1
        if total >= test_generator.samples:
            break

    accuracy = correct / total
    print(f"TFLite Model - Test Accuracy: {accuracy:.4f}")

# Main execution
if __name__ == "__main__":
    print("Evaluating Keras model...")
    evaluate_keras_model()
    print("\nEvaluating TFLite model...")
    evaluate_tflite_model()

Evaluating Keras model...


Found 1667 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


53/53 ━━━━━━━━━━━━━━━━━━━━ 9s 145ms/step - accuracy: 0.9768 - loss: 0.1503
Keras Model - Test Loss: 0.1300, Test Accuracy: 0.9802

Evaluating TFLite model...
Found 1667 images belonging to 2 classes.
TFLite Model - Test Accuracy: 0.9802
